<a href="https://colab.research.google.com/github/shazzad-hasan/training-reproducible-deep-learning-models/blob/main/AlexNet/alexnet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shazzad-hasan/training-reproducable-deep-learning-models.git

Cloning into 'training-reproducable-deep-learning-models'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 126 (delta 51), reused 73 (delta 26), pack-reused 0
Receiving objects: 100% (126/126), 30.96 KiB | 905.00 KiB/s, done.
Resolving deltas: 100% (51/51), done.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd /content/training-reproducable-deep-learning-models/AlexNet

/content/training-reproducable-deep-learning-models/AlexNet


In [4]:
!ls

helper_dataset.py  helper_evaluate.py  helper_plot.py  helper_train.py


In [5]:
# import required libraries
import torch
import torchvision 

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import local helper functions
